# クラス構成によるデータ構造の改善

このノートブックでは、深くネストした辞書の代わりにクラスを使用してデータ構造を改善する方法を学びます。


In [ ]:
# ネストした辞書の問題とクラスによる解決

# 悪い例（深くネストした辞書）
def create_user_data_bad():
    """ユーザーデータを作成（ネストした辞書）"""
    return {
        'user': {
            'id': 1,
            'name': 'Alice',
            'profile': {
                'age': 25,
                'email': 'alice@example.com',
                'address': {
                    'street': '123 Main St',
                    'city': 'Tokyo',
                    'country': 'Japan'
                }
            },
            'settings': {
                'theme': 'dark',
                'notifications': {
                    'email': True,
                    'push': False
                }
            }
        }
    }

# 良い例（クラスを使用）
class Address:
    """住所クラス"""
    def __init__(self, street, city, country):
        self.street = street
        self.city = city
        self.country = country
    
    def __str__(self):
        return f"{self.street}, {self.city}, {self.country}"

class Notifications:
    """通知設定クラス"""
    def __init__(self, email=True, push=False):
        self.email = email
        self.push = push

class Settings:
    """設定クラス"""
    def __init__(self, theme='light', notifications=None):
        self.theme = theme
        self.notifications = notifications or Notifications()

class Profile:
    """プロフィールクラス"""
    def __init__(self, age, email, address):
        self.age = age
        self.email = email
        self.address = address

class User:
    """ユーザークラス"""
    def __init__(self, user_id, name, profile, settings):
        self.id = user_id
        self.name = name
        self.profile = profile
        self.settings = settings
    
    def get_full_address(self):
        """完全な住所を取得"""
        return str(self.profile.address)
    
    def is_notification_enabled(self, type_name):
        """通知が有効かどうか"""
        return getattr(self.settings.notifications, type_name, False)

# 使用例
print("=== ネストした辞書の問題 ===")
nested_data = create_user_data_bad()
print(f"ネストした辞書: {nested_data}")

print("\n=== クラスによる解決 ===")
address = Address('123 Main St', 'Tokyo', 'Japan')
notifications = Notifications(email=True, push=False)
settings = Settings(theme='dark', notifications=notifications)
profile = Profile(25, 'alice@example.com', address)
user = User(1, 'Alice', profile, settings)

print(f"ユーザー: {user.name}")
print(f"住所: {user.get_full_address()}")
print(f"メール通知: {user.is_notification_enabled('email')}")
print(f"プッシュ通知: {user.is_notification_enabled('push')}")


In [ ]:
# データベース設定でのクラス活用

class DatabaseConnection:
    """データベース接続クラス"""
    def __init__(self, host, port, database, username, password):
        self.host = host
        self.port = port
        self.database = database
        self.username = username
        self.password = password
    
    def get_connection_string(self):
        """接続文字列を取得"""
        return f"postgresql://{self.username}:{self.password}@{self.host}:{self.port}/{self.database}"
    
    def get_pool_settings(self):
        """プール設定を取得"""
        return {
            'min_connections': 1,
            'max_connections': 10,
            'timeout': 30
        }

class DatabaseConfig:
    """データベース設定クラス"""
    def __init__(self, connection, pool_settings=None):
        self.connection = connection
        self.pool_settings = pool_settings or {}
    
    def validate(self):
        """設定を検証"""
        if not self.connection.host:
            raise ValueError("ホストが設定されていません")
        if not self.connection.database:
            raise ValueError("データベース名が設定されていません")
        return True

# 使用例
print("=== データベース設定 ===")
connection = DatabaseConnection('localhost', 5432, 'myapp', 'postgres', 'password')
config = DatabaseConfig(connection)

try:
    config.validate()
    print(f"接続文字列: {connection.get_connection_string()}")
    print(f"プール設定: {connection.get_pool_settings()}")
except ValueError as e:
    print(f"設定エラー: {e}")

# API設定でのクラス活用
class APITimeout:
    """APIタイムアウト設定クラス"""
    def __init__(self, connect=10, read=30, total=60):
        self.connect = connect
        self.read = read
        self.total = total

class APIRetry:
    """APIリトライ設定クラス"""
    def __init__(self, max_retries=3, delay=1, backoff_factor=2):
        self.max_retries = max_retries
        self.delay = delay
        self.backoff_factor = backoff_factor

class APIConfig:
    """API設定クラス"""
    def __init__(self, base_url, timeout=None, retry=None, headers=None):
        self.base_url = base_url
        self.timeout = timeout or APITimeout()
        self.retry = retry or APIRetry()
        self.headers = headers or {}
    
    def get_full_url(self, endpoint):
        """完全なURLを取得"""
        return f"{self.base_url.rstrip('/')}/{endpoint.lstrip('/')}"
    
    def get_headers(self):
        """ヘッダーを取得"""
        return {
            'Content-Type': 'application/json',
            **self.headers
        }

# 使用例
print("\n=== API設定 ===")
timeout = APITimeout(connect=15, read=45, total=90)
retry = APIRetry(max_retries=5, delay=2, backoff_factor=3)
api_config = APIConfig(
    base_url='https://api.example.com',
    timeout=timeout,
    retry=retry,
    headers={'Authorization': 'Bearer token123'}
)

print(f"完全なURL: {api_config.get_full_url('/users')}")
print(f"ヘッダー: {api_config.get_headers()}")
print(f"タイムアウト設定: {api_config.timeout.__dict__}")
print(f"リトライ設定: {api_config.retry.__dict__}")
